In [1]:
import pandas as pd
from intertrans.data import read_engine_output
import itertools
import numpy as np
from collections import defaultdict
import pandas as pd

### Data sampling for qualitative analysis (RQ3 and Supplementary Material)

In [2]:
df_codellama_13b_transcoder_noverify = read_engine_output('../data/raw_outputs/engine/codellama_13b_transcoder_results_all_depth4.json')
df_magicoder_transcoder_noverify = read_engine_output('../data/raw_outputs/engine/magicoder_transcoder_results_all_depth4.json')
df_starcoder2_transcoder_noverify = read_engine_output('../data/raw_outputs/engine/starcoder2_transcoder_results_all_depth4.json')

In [3]:
df_codellama_13b_humanevalx_noverify = read_engine_output('../data/raw_outputs/engine/codellama_13b_humanevalx_results_sub_depth4.json')
df_magicoder_humanevalx_noverify = read_engine_output('../data/raw_outputs/engine/magicoder_humanevalx_results_sub_depth4.json')
df_starcoder2_humanevalx_noverify = read_engine_output('../data/raw_outputs/engine/starcoder2_humanevalx_results_sub_depth4.json')

In [4]:
df_codellama_13b_codenet_noverify = read_engine_output('../data/raw_outputs/engine/codellama_13b_codenet_results_sub_depth4.json')
df_magicoder_codenet_noverify = read_engine_output('../data/raw_outputs/engine/magicoder_codenet_results_sub_depth4.json')
df_starcoder2_codenet_noverify = read_engine_output('../data/raw_outputs/engine/starcoder2_codenet_results_sub_depth4.json')

In [46]:
transcoder_dataset_all = pd.read_json('../datasets/transcoder_dataset_all.jsonl', orient='records', lines=True)
humanevalx_subset = pd.read_json('../datasets/humanevalx_dataset_subset.jsonl', orient='records', lines=True)
codenet_subset = pd.read_json('../datasets/codenet_dataset_subset.jsonl', orient='records', lines=True)

In [58]:
humanevalx_subset.head()

,id,source_lang,input_code,target_lang,ground_truth,target_signature,test_code
0,138,C++,#include<stdio.h>\n#include<math.h>\n#include<...,Go,\nfunc IsEqualToSumEven(n int) bool {\n ret...,\nfunc IsEqualToSumEven(n int) bool {\n,func TestIsEqualToSumEven(t *testing.T) {\n ...
1,140,C++,#include<stdio.h>\n#include<string>\nusing nam...,Go,\nfunc FixSpaces(text string) string {\n ne...,\nfunc FixSpaces(text string) string {\n,func TestFixSpaces(t *testing.T) {\n assert...
2,98,C++,#include<stdio.h>\n#include<math.h>\n#include<...,Go,\nfunc CountUpper(s string) int {\n count :...,\nfunc CountUpper(s string) int {\n,func TestCountUpper(t *testing.T) {\n asser...
3,158,C++,#include<stdio.h>\n#include<vector>\n#include<...,Go,\nfunc FindMax(words []string) string {\n k...,\nfunc FindMax(words []string) string {\n,func TestFindMax(t *testing.T) {\n assert :...
4,93,C++,#include<stdio.h>\n#include<math.h>\n#include<...,Go,\nfunc Encode(message string) string {\n vo...,\nfunc Encode(message string) string {\n,func TestEncode(t *testing.T) {\n assert :=...


In [59]:
codenet_subset.head()

,task_id,source_lang,input_code,target_lang,ground_truth,stdin_input_1,expected_output_1,stdin_input_2,expected_output_2,stdin_input_3,expected_output_3
0,p03062,C++,"#include <bits/stdc++.h>\n#define REP(i, s, n)...",Go,"package main\n\nimport (\n\t""fmt""\n\t""math""\n)...",3\n-10 5 -4\n,19,5\n10 -4 -8 -11 3\n,30,11\n-1000000000 1000000000 -1000000000 1000000...,10000000000
1,p03385,C++,#include<bits/stdc++.h>\nusing namespace std;\...,Go,"package main\n\nimport (\n\t""fmt""\n\t""strings""...",bac\n,Yes,bab\n,No,abc\n,Yes
2,p03543,C++,"#include <bits/stdc++.h>\n#define rep(i,m,n) f...",Go,"package main\n\nimport ""fmt""\n\nfunc main(){\n...",1118\n,Yes,7777\n,Yes,1234\n,No
3,p03324,C++,#include <iostream>\n#include <cmath>\nusing n...,Go,///\n// File: b.go\n// Author: ymiyamoto\n//\...,0 5\n,5,1 11\n,1100,2 85\n,850000
4,p03433,C++,#include <iostream>\nusing namespace std;\n\ni...,Go,"package main\n\nimport (\n\t""fmt""\n)\n\nfunc m...",2018\n218\n,Yes,2763\n0\n,No,37\n514\n,Yes


In [60]:
data_dict = {
    'humanevalx': {
        'magicoder': {
            'noverify': df_magicoder_humanevalx_noverify,
            'dataset' : humanevalx_subset
        },
        'codellama_13b': {
            'noverify': df_codellama_13b_humanevalx_noverify,
            'dataset' : humanevalx_subset
        },
        'starcoder2': {
            'noverify': df_starcoder2_humanevalx_noverify,
            'dataset' : humanevalx_subset
        }
    },
    'codenet': {
        'magicoder': {
            'noverify': df_magicoder_codenet_noverify,
            'dataset' : codenet_subset
        },
        'codellama_13b': {
            'noverify': df_codellama_13b_codenet_noverify,
            'dataset' : codenet_subset
        },
        'starcoder2': {
            'noverify': df_starcoder2_codenet_noverify,
            'dataset' : codenet_subset
        }
    },
    'transcoder': {
        'magicoder': {
            'noverify': df_magicoder_transcoder_noverify,
            'dataset' : transcoder_dataset_all            
        },
        'codellama_13b': {
            'noverify': df_codellama_13b_transcoder_noverify,
            'dataset' : transcoder_dataset_all
        },
        'starcoder2': {
            'noverify': df_starcoder2_transcoder_noverify,
            'dataset' : transcoder_dataset_all
        }
    }
}

In [61]:
import copy

# Function to perform ablation on programming languages combinations
def flatten_paths(raw_json, ignore_direct=False):
    samples = []
    # Iterate through each translation response in the raw JSON data
    for response in raw_json['translation_responses']:
        seed_language = response['translation_request']['seed_language']
        target_language = response['translation_request']['target_language']
        id_request = response['translation_request']['id']
        seed_code = response['translation_request']['seed_code']

        if ignore_direct:
            firstEdge = response['paths'][0]["translation_edges"][0]
            if firstEdge['target_language'] == target_language and firstEdge['input_language'] == seed_language and firstEdge['status'] == "TRANSLATION_FOUND":
                continue

        path_id = 0
        for path in response['paths']:
            found = False
            languages = []
            extracted_code = []
            unit_tests_list = []
            executed_code = []
            inference_outputs = []
            prompts = []

            for edge in path["translation_edges"]:
                languages.append(edge['target_language'])
                extracted_code.append(edge.get('extracted_source_code', ""))
                inference_outputs.append(edge.get('inference_output', ""))
                prompts.append(edge.get('prompt', ""))
                

                if edge['status'] == "FAILED_EXECUTION" or edge['status'] == "FAILED_VERIFICATION" or edge['status'] == "FAILED" or edge['status'] == "TRANSLATION_FOUND":
                    if edge['status'] == "TRANSLATION_FOUND":
                        found = True
                
                    if "unit_tests" in edge.keys():
                        tests = edge["unit_tests"]
                    elif "fuzzy_tests" in edge.keys():
                        tests = edge["fuzzy_tests"]
                    else:
                        tests = []

                    for test in tests:
                        output = test.get("actual_output", "")
                        exec_code = test.get("executed_code", "")
                        unit_tests_list.append(output)
                        executed_code.append(exec_code) 

            obj = {}
            obj['path_id'] = path_id
            obj['id_request'] = id_request
            obj['seed_lang'] = seed_language
            obj['target_language'] = target_language
            obj['seed_code'] = seed_code
            obj['prompts'] = prompts
            obj['intermediates'] = languages
            obj['extracted_code'] = extracted_code
            obj['status'] = edge['status']
            obj['success'] = found
            obj['tests_output'] = unit_tests_list
            obj['executed_test_code'] = executed_code
            obj['inference_outputs'] = inference_outputs

            samples.append(obj)

            path_id+=1

    df = pd.DataFrame(samples)
    return df


In [62]:
def find_ground_truth(df, row):
    found = df[(df.source_lang == row['seed_lang']) & (df.target_lang == row['target_language']) & (df.input_code == row['seed_code'])]
    if found.empty:
        print("Weird")
    gt = found.iloc[0]["ground_truth"]
    return gt


In [63]:
def get_experiments_sample(data_dict):
    # Extract values and construct the multi-level index
    results_df = pd.DataFrame()

    for evaluation_type, verify_dict in data_dict.items():
        for model_name, verify_status_dict in verify_dict.items():
            print(evaluation_type)
            #Keep only the cases where a direct translation was not found but yes through InterTrans
            df = flatten_paths(verify_status_dict['noverify'])
            dataset = verify_status_dict['dataset']
            samples = []

            for name, group in df.groupby(by="id_request"):
                direct_translation_failed = group[(group['intermediates'].apply(len) == 1) & (group['success'] == False)]

                if not direct_translation_failed.empty:
                    assert len(direct_translation_failed) == 1
                    success_paths = group[(group['success'] == True)]
                    
                    if not success_paths.empty:
                        failed_row = direct_translation_failed.iloc[0].copy()
                        failed_row['ground_truth'] = find_ground_truth(dataset, failed_row)
                        samples.append(failed_row)
                        for i, row in success_paths.iterrows():
                            #We take the first successful path following the algorithm
                            success_row = row.copy()
                            success_row['ground_truth'] = find_ground_truth(dataset, success_row)
                            samples.append(success_row)
                            break
                        

            filtered = pd.DataFrame(samples)

            # Sample the group identifiers
            unique_groups = filtered['id_request'].unique()
            sampled_groups = pd.Series(unique_groups).sample(n=50, random_state=1)

            # Filter the DataFrame based on the sampled group identifiers
            sampled_df = filtered[filtered['id_request'].isin(sampled_groups)].copy()
            sampled_df["code_length"] = sampled_df['seed_code'].apply(lambda x: len(x))

            if sampled_df.shape[0] != sampled_groups.shape[0] * 2:
                print("Wrong")

            # Create DataFrames for each element of the arrays and concatenate
            extracted_code_df = sampled_df['extracted_code'].apply(pd.Series)
            extracted_code_df.columns = [f'extracted_code_{i+1}' for i in range(extracted_code_df.shape[1])]

            executed_test_code_df = sampled_df['executed_test_code'].apply(pd.Series)
            executed_test_code_df.columns = [f'executed_test_code_{i+1}' for i in range(executed_test_code_df.shape[1])]

            inference_outputs_df = sampled_df['inference_outputs'].apply(pd.Series)
            inference_outputs_df.columns = [f'inference_outputs_{i+1}' for i in range(inference_outputs_df.shape[1])]

            tests_outputs_df = sampled_df['tests_output'].apply(pd.Series)
            tests_outputs_df.columns = [f'tests_output_{i+1}' for i in range(tests_outputs_df.shape[1])]

            prompt_df = sampled_df['prompts'].apply(pd.Series)
            prompt_df.columns = [f'prompts_{i+1}' for i in range(prompt_df.shape[1])]

            # Concatenate the original DataFrame with the new DataFrames
            df_expanded = pd.concat([sampled_df.drop(columns=['extracted_code', 'executed_test_code', 'inference_outputs', 'tests_output', 'prompts']),
                                    extracted_code_df, executed_test_code_df, inference_outputs_df, tests_outputs_df, prompt_df], axis=1)

            df_expanded["model"] = model_name
            df_expanded["dataset"] = evaluation_type

            # Concatenate the results DataFrame with the new DataFrame
            results_df = pd.concat([results_df, df_expanded])

    return results_df


In [64]:
qualitative_samples = get_experiments_sample(data_dict)

humanevalx
humanevalx
humanevalx
codenet
codenet
codenet
transcoder
transcoder
transcoder


In [65]:
qualitative_samples.head().T

,1955,1956,2380,2381,7310
path_id,0,1,0,1,0
id_request,1018,1018,1022,1022,130
seed_lang,Rust,Rust,Rust,Rust,C++
target_language,Python,Python,Python,Python,Python
seed_code,"\nuse std::{slice::Iter, cmp::{max, self}, mem...","\nuse std::{slice::Iter, cmp::{max, self}, mem...","\nuse std::{slice::Iter, cmp::{max, self}, mem...","\nuse std::{slice::Iter, cmp::{max, self}, mem...",#include<stdio.h>\n#include<vector>\nusing nam...
intermediates,[Python],"[JavaScript, Python]",[Python],"[JavaScript, Python]",[Python]
status,FAILED_EXECUTION,TRANSLATION_FOUND,FAILED_EXECUTION,TRANSLATION_FOUND,FAILED_EXECUTION
success,False,True,False,True,False
ground_truth,"def find_max(words):\n return sorted(words,...","def find_max(words):\n return sorted(words,...","def greatest_common_divisor(a: int, b: int) ->...","def greatest_common_divisor(a: int, b: int) ->...",def get_max_triples(n):\n A = [i*i - i + 1 ...
code_length,706,706,319,319,673


In [66]:
qualitative_samples.shape

(900, 30)

In [67]:
qualitative_samples.to_excel('rq4_samples.xlsx', index=True, engine='xlsxwriter')